In [26]:
import os
from dotenv import load_dotenv

def ensure_api_keys() -> tuple[str, str]:

    load_dotenv()

    required_vars = ["GOOGLE_API_KEY", "TAVILY_API_KEY"]
    missing = []

    for var in required_vars:
        if not os.environ.get(var):
            missing.append(var)

    if missing:
        missing_list = ", ".join(missing)
        raise EnvironmentError(
            f"Missing required environment variables: {missing_list}. "
            "Set them in your OS environment or .env file."
        )

    return os.environ["GOOGLE_API_KEY"], os.environ["TAVILY_API_KEY"]


GOOGLE_API_KEY, TAVILY_API_KEY = ensure_api_keys()


# **1. What `init_chat_model` Actually Does**

`init_chat_model` is a **unified factory** that:

1. Detects **which LLM provider** you want (OpenAI, Anthropic, Google, AWS, etc.)
2. Loads the correct **provider-specific LangChain wrapper**
3. Instantiates the model with the parameters you pass (temperature, max_tokens, etc.)
4. Optionally builds a **runtime-configurable model** (model chosen dynamically per request)
5. Returns a **BaseChatModel** object with standard methods:

   * `.invoke()` — single request
   * `.stream()` — streaming tokens
   * `.batch()` — multiple requests at once

This means you can write **one unified LangChain agent**, and switch models with zero code changes.



In [27]:
from langchain.chat_models import init_chat_model

model = init_chat_model(
    "google_genai:gemini-2.5-flash-lite",
    temperature=0.2,
    max_tokens=1024
)

response = model.invoke("Why do parrots talk?")
print(response.content)

Parrots talk for a fascinating combination of reasons, driven by their **intelligence, social nature, and evolutionary adaptations.** It's not just about mimicking sounds; it's a complex behavior with multiple underlying motivations.

Here's a breakdown of why parrots talk:

**1. Social Bonding and Communication:**

*   **Flock Behavior:** Parrots are highly social animals that live in flocks in the wild. Communication is crucial for maintaining group cohesion, warning of danger, finding food, and coordinating activities. Talking, or vocalizing, is their primary way of doing this.
*   **Mimicry as a Social Tool:** In the wild, parrots often mimic the calls of other birds in their flock. This helps them blend in, identify members, and understand the social dynamics of their group. When they mimic human sounds, it's essentially an extension of this innate social behavior. They are trying to integrate into their "flock" (their human family).

**2. Intelligence and Cognitive Abilities:**




# **2. Full Parameter Breakdown**
# **`model`**

### **Purpose**

The model ID you want to use, optionally including the provider name.

### **Accepted formats**

1. `"gpt-4o"`
2. `"openai:gpt-4o"`
3. `"google_genai:gemini-2.5-flash-lite"`
4. `"anthropic:claude-3.5-sonnet"`

# **`configurable_fields`**
Controls which parameters can be changed **at runtime**.
### Modes
| Value      | Meaning                                          |
| ---------- | ------------------------------------------------ |
| `None`     | Model is fixed after creation                    |
| `"any"`    | ALL fields configurable (dangerous for security) |
| List/Tuple | Only selected parameters configurable            |

### **When this is useful**
* Building a **UI allowing users to pick models**
* Making an **API gateway** that routes dynamically
* Experimenting with model sweeps


In [28]:
init_chat_model(
    "gpt-4o",
    configurable_fields=("temperature", "model")
)

In [29]:
# Then at runtime:
model.invoke(
    "hello",
    config={ "configurable": { "temperature": 0.9 } }
)

AIMessage(content='Hello there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--586fb4bd-7132-4388-8391-2e2ba1a93c0a-0', usage_metadata={'input_tokens': 2, 'output_tokens': 10, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})


# **`config_prefix`**

Adds a prefix to runtime configuration keys.

### **Why this matters**

If multiple configurable models exist in the same agent pipeline, prefixes prevent collision.

---

# **`**kwargs` — Provider-specific parameters**

This is where you pass typical generation parameters.

### Standard kwargs (supported mostly everywhere):

| Param         | Meaning                  |
| ------------- | ------------------------ |
| `temperature` | randomness               |
| `max_tokens`  | max output tokens        |
| `timeout`     | max request time         |
| `max_retries` | retry attempts           |
| `api_key`     | override environment key |
| `base_url`    | custom server endpoint   |



In [30]:

configurable_model = init_chat_model(
    "gpt-4o",
    configurable_fields="any",
    config_prefix="foo"
)


In [31]:
model.invoke(
    "hello",
    config={ "configurable": { "foo_temperature": 0.8 } }
)

AIMessage(content='Hello there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--b7447e91-1494-481d-9bd5-3f3e8a6b6f36-0', usage_metadata={'input_tokens': 2, 'output_tokens': 10, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

In [32]:

init_chat_model(
    "google_genai:gemini-2.5-flash-lite",
    temperature=0.4,
    max_tokens=2048,
    timeout=20,
    max_retries=2
)


ChatGoogleGenerativeAI(profile={'max_input_tokens': 1048576, 'max_output_tokens': 65536, 'image_inputs': True, 'audio_inputs': True, 'pdf_inputs': True, 'video_inputs': True, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'image_tool_message': True, 'tool_choice': True}, model='models/gemini-2.5-flash-lite', google_api_key=SecretStr('**********'), temperature=0.4, max_output_tokens=2048, max_retries=2, timeout=20.0, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000025B197E7440>, default_metadata=(), model_kwargs={})


# **Return Type**

`init_chat_model` returns one of two things:

### **A. Normal fixed model**

→ When `model` is supplied and `configurable_fields=None`.

### **B. Configurable model wrapper**

→ When `model` missing OR `configurable_fields` is used.

This wrapper defers calculation of:

* Model provider
* Model name
* All kwargs

Until `invoke()` is called.



In [ ]:
## **Mode 1 — Fixed Model (most common)**

model = init_chat_model("google_genai:gemini-2.5-flash-lite",
temperature=0)
model.invoke("hello")


AIMessage(content='Hello there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--95e9cceb-71e2-440d-93b4-58207de056ed-0', usage_metadata={'input_tokens': 2, 'output_tokens': 10, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

In [35]:
## **Mode 2 — Partially Configurable Model**

model = init_chat_model(temperature=0)
model.invoke(
    "hi",
    config={"configurable": {"model": "google_genai:gemini-2.5-flash-lite"}}
)



AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--b8e7cfd6-2061-40d4-beae-2bcf959f202b-0', usage_metadata={'input_tokens': 2, 'output_tokens': 10, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

In [45]:

## **Mode 3 — Fully Configurable Model**

model = init_chat_model(
    "google_genai:gemini-2.5-flash-lite",
    configurable_fields="any",
    config_prefix="foo"
)


config = {
    "configurable": {
        "foo_model": "google_genai:gemini-2.5-flash-lite",
        "foo_temperature": 0.8
    }
}
model.invoke("hello", config=config)


AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--1924013d-c7f3-43ba-9d4e-18524a86142d-0', usage_metadata={'input_tokens': 2, 'output_tokens': 10, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

In [ ]:

## **Mode 4 — Tools + Configurable Model**

model = init_chat_model(
    "google_genai:gemini-2.5-flash-lite",
    configurable_fields=("model", "model_provider")
)

tool_model = model.bind_tools([GetWeather, GetPopulation])

tool_model.invoke("LA or NY?", config={"configurable": {"model": "claude-3.5"}})

NameError: name 'GetWeather' is not defined

In [40]:
from langchain_tavily import TavilySearch  # updated at 1.0

tavily_search = TavilySearch(max_results=5)

data = tavily_search.invoke({"query": "What is LangGraph?"})
search_docs = data.get("results", data)


In [41]:
search_docs

[{'url': 'https://www.datacamp.com/tutorial/langgraph-tutorial',
  'title': 'LangGraph Tutorial: What Is LangGraph and How to Use It?',
  'content': 'LangGraph is a library within the LangChain ecosystem that provides a framework for defining, coordinating, and executing multiple LLM agents (or chains) in a structured and efficient manner. By managing the flow of data and the sequence of operations, LangGraph allows developers to focus on the high-level logic of their applications rather than the intricacies of agent coordination. Whether you need a chatbot that can handle various types of user requests or a multi-agent system that performs complex tasks, LangGraph provides the tools to build exactly what you need. LangGraph significantly simplifies the development of complex LLM applications by providing a structured framework for managing state and coordinating agent interactions.',
  'score': 0.9581988,
  'raw_content': None},
 {'url': 'https://www.geeksforgeeks.org/machine-learning

In [46]:
from langchain_core.messages import HumanMessage

# Create a message
msg = HumanMessage(content="Hello world", name="Lance")

# Message list
messages = [msg]

model.invoke(messages)

AIMessage(content='Hello to you too! How can I help you today? 😊', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--e7ca0edc-8f08-4ff7-87fc-c81a84ce6c82-0', usage_metadata={'input_tokens': 3, 'output_tokens': 13, 'total_tokens': 16, 'input_token_details': {'cache_read': 0}})